In [47]:
!pip install numba numpy

In [48]:
import numpy as np
from numba import cuda, float32

@cuda.jit
def dotProd(d_a, d_b, d_dotprod, N):
    sdata = cuda.shared.array(32, dtype=float32)  # Shared memory for parallel reduction

    tx = cuda.threadIdx.x
    bx = cuda.blockIdx.x
    bw = cuda.blockDim.x

    start = tx
    stride = bw

    dot_product = 0.0

    # Compute element-wise products and perform parallel reduction in shared memory
    while start < N:
        dot_product += d_a[start] * d_b[start]
        start += stride

    sdata[tx] = dot_product

    # Synchronize !!
    cuda.syncthreads()

    # parallel reduction in shared memory
    i = cuda.blockDim.x // 2
    while i != 0:
        if tx < i:
            sdata[tx] += sdata[tx + i]
        cuda.syncthreads()
        i //= 2

    # Store the final result in global memory
    if tx == 0:
        d_dotprod[bx] = sdata[0]


if __name__ == "__main__":
    N = 5

    # Allocate host memory
    h_a = np.random.randint(1, 11, N).astype(np.float32)
    h_b = np.random.randint(1, 11, N).astype(np.float32)
    h_dotprod = np.zeros(1, dtype=np.float32)

    # Allocate device memory
    d_a = cuda.to_device(h_a)
    d_b = cuda.to_device(h_b)
    d_dotprod = cuda.device_array(1, dtype=np.float32)

    # Launch kernel
    block_size = 32
    grid_size = (N + block_size - 1) // block_size
    dotProd[grid_size, block_size](d_a, d_b, d_dotprod, N)

    # Copy result to host
    d_dotprod.copy_to_host(h_dotprod)

    # Print result
    print("A:", h_a)
    print("B:", h_b)
    print("Dot Product:", h_dotprod[0])


A: [ 8. 10.  7.  9.  1.]
B: [5. 9. 8. 6. 8.]
Dot Product: 248.0


/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


In [49]:
import numpy as np
from numba import cuda, float32
import math

@cuda.jit(device=True)
def block_reduce(a):
    # Perform parallel reduction within a block
    shared_val = cuda.shared.array(32, dtype=float32)
    tid = cuda.threadIdx.x
    bid = cuda.blockIdx.x
    bdim = cuda.blockDim.x

    shared_val[tid] = a
    cuda.syncthreads()

    # Perform reduction in shared memory
    for s in range(bdim // 2):
        if tid < bdim // 2:
            shared_val[tid] += shared_val[tid + s + 1]
        cuda.syncthreads()

    return shared_val[0]

@cuda.jit
def mean_var_norm(d_a, d_mean, d_var, d_norm, N):
    col = cuda.threadIdx.x
    stride = cuda.blockDim.x

    # Compute mean using parallel reduction(PR)
    mean_val = 0.0
    for i in range(col, N, stride):
        mean_val += d_a[i]

    mean_val = cuda.atomic.add(d_mean, 0, mean_val)
    cuda.syncthreads()

    # Compute variance using PR
    var_val = 0.0
    for i in range(col, N, stride):
        var_val += (d_a[i] - d_mean[0]) ** 2

    var_val = cuda.atomic.add(d_var, 0, var_val)
    cuda.syncthreads()

    # Synchronize to make sure mean and variance are calculated
    cuda.syncthreads()

    # Compute normalized values
    for i in range(col, N, stride):
        d_norm[i] = (d_a[i] - d_mean[0]) / (math.sqrt(d_var[0] / N + 1e-8))



if __name__ == "__main__":
    N = 5

    # Allocate host memory
    h_a = np.random.randint(1, 11, N).astype(np.float32)
    h_mean = np.zeros(1, dtype=np.float32)
    h_var = np.zeros(1, dtype=np.float32)
    h_norm = np.zeros(N, dtype=np.float32)

    # Allocate device memory
    d_a = cuda.to_device(h_a)
    d_mean = cuda.device_array(1, dtype=np.float32)
    d_var = cuda.device_array(1, dtype=np.float32)
    d_norm = cuda.device_array(N, dtype=np.float32)

    # Launch kernel
    block_size = 32
    grid_size = (N + block_size - 1) // block_size
    mean_var_norm[grid_size, block_size](d_a, d_mean, d_var, d_norm, N)

    # Copy results to host
    d_mean.copy_to_host(h_mean)
    d_var.copy_to_host(h_var)
    d_norm.copy_to_host(h_norm)


    print("A:", h_a)
    print("Mean:", h_mean[0])
    print("Variance:", h_var[0])
    print("Normalized Values:", h_norm)


/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


A: [8. 3. 1. 6. 2.]
Mean: 20.0
Variance: 1314.0
Normalized Values: [-0.7402332  -1.0486637  -1.1720359  -0.86360544 -1.1103498 ]


In [50]:
import numpy as np
from numba import cuda, float32

# Matrix size
M, N = 512, 512

@cuda.jit
def matAdd(d_A, d_B, d_C):
    # Allocate shared memory for matrices A, B, and C
    s_A = cuda.shared.array(shape=(16, 16), dtype=float32)
    s_B = cuda.shared.array(shape=(16, 16), dtype=float32)
    s_C = cuda.shared.array(shape=(16, 16), dtype=float32)

    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    bx = cuda.blockIdx.x
    by = cuda.blockIdx.y

    # Load elements into shared memory
    row_A = by * 16 + ty
    col_A = bx * 16 + tx
    row_B = by * 16 + ty
    col_B = bx * 16 + tx

    s_A[ty, tx] = d_A[row_A, col_A]
    s_B[ty, tx] = d_B[row_B, col_B]

    cuda.syncthreads()

    # Perform matrix addition in shared memory
    s_C[ty, tx] = s_A[ty, tx] + s_B[ty, tx]

    cuda.syncthreads()

    # Store the result in global memory
    row_C = by * 16 + ty
    col_C = bx * 16 + tx
    d_C[row_C, col_C] = s_C[ty, tx]


if __name__ == "__main__":

    h_A = np.random.rand(M, N).astype(np.float32)
    h_B = np.random.rand(M, N).astype(np.float32)
    h_C = np.empty_like(h_A)

    # Allocate device memory
    d_A = cuda.to_device(h_A)
    d_B = cuda.to_device(h_B)
    d_C = cuda.device_array_like(h_C)

    # Define block and grid dimensions
    block_dim = (16, 16)
    grid_dim = ((N + block_dim[1] - 1) // block_dim[1], (M + block_dim[0] - 1) // block_dim[0])

    # Launch kernel
    matAdd[grid_dim, block_dim](d_A, d_B, d_C)

    # Copy result to host
    d_C.copy_to_host(h_C)

    # Display results
    print("Matrix A:")
    print(h_A)
    print("----------")
    print("Matrix B:")
    print(h_B)
    print("----------")
    print("Matrix C (Result of Addition):")
    print(h_C)


Matrix A:
[[0.633758   0.33405107 0.5288251  ... 0.5341994  0.94313174 0.87808603]
 [0.99936557 0.4754105  0.7203953  ... 0.8777755  0.8494717  0.6279307 ]
 [0.1533908  0.778205   0.44190192 ... 0.9093095  0.35668865 0.81189936]
 ...
 [0.6389082  0.73845655 0.6936545  ... 0.09694602 0.09927858 0.68780136]
 [0.5997956  0.8147846  0.06913508 ... 0.8301413  0.9555761  0.49095625]
 [0.5183593  0.7990846  0.5602129  ... 0.8253773  0.3116791  0.6956825 ]]
----------
Matrix B:
[[0.82938707 0.7934714  0.8981924  ... 0.41581234 0.9609986  0.15894851]
 [0.35991558 0.9129847  0.74222845 ... 0.25218287 0.5928586  0.50405324]
 [0.7566857  0.8522018  0.476235   ... 0.12081056 0.49309912 0.98133796]
 ...
 [0.30231717 0.8695147  0.33973026 ... 0.36438474 0.39720768 0.34914818]
 [0.06269992 0.5950702  0.64461607 ... 0.793343   0.9580506  0.32525393]
 [0.30928665 0.14187622 0.05999177 ... 0.82270384 0.97562814 0.73048717]]
----------
Matrix C (Result of Addition):
[[1.463145   1.1275225  1.4270175  ... 

In [51]:
import numpy as np
from numba import cuda, float32

@cuda.jit
def matMul(d_A, d_B, d_C):
    row, col = cuda.grid(2)

    if row < d_C.shape[0] and col < d_C.shape[1]:
        M, N = d_A.shape
        P = d_B.shape[1]
        sum = 0.0

        for i in range(N):
            sum += d_A[row, i] * d_B[i, col]

        d_C[row, col] = sum

if __name__ == "__main__":

    M, N, P = 3, 4, 2


    h_A = np.random.randint(1, 10, size=(M, N)).astype(np.float32)
    h_B = np.random.randint(1, 10, size=(N, P)).astype(np.float32)
    h_C = np.empty((M, P), dtype=np.float32)

    # Allocate device memory
    d_A = cuda.to_device(h_A)
    d_B = cuda.to_device(h_B)
    d_C = cuda.device_array_like(h_C)

    block_dim = (16, 16)
    grid_dim = ((P + block_dim[1] - 1) // block_dim[1], (M + block_dim[0] - 1) // block_dim[0])

    matMul[grid_dim, block_dim](d_A, d_B, d_C)

    # Copy result to host
    d_C.copy_to_host(h_C)


    print("Matrix A:")
    print(h_A)
    print("--------")
    print("Matrix B:")
    print(h_B)
    print("--------")
    print("Matrix C (Result of Multiplication):")
    print(h_C)


/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


Matrix A:
[[5. 9. 1. 5.]
 [6. 5. 2. 8.]
 [3. 1. 9. 5.]]
--------
Matrix B:
[[7. 4.]
 [2. 9.]
 [5. 1.]
 [7. 5.]]
--------
Matrix C (Result of Multiplication):
[[ 93. 127.]
 [118. 111.]
 [103.  55.]]


In [52]:
import numpy as np
from numba import cuda, float32

@cuda.jit
def matScale(d_A, d_B, scale):
    row, col = cuda.grid(2)

    if row < d_B.shape[0] and col < d_B.shape[1]:
        d_B[row, col] = d_A[row, col] / scale

if __name__ == "__main__":

    M, N = 4, 4


    scale = 2.5


    h_A = np.random.randint(1, 10, size=(M, N)).astype(np.float32)
    h_B = np.empty_like(h_A)

    # Allocate device memory
    d_A = cuda.to_device(h_A)
    d_B = cuda.to_device(h_B)

    # Define block and grid dimensions
    block_dim = (16, 16)
    grid_dim = ((N + block_dim[1] - 1) // block_dim[1], (M + block_dim[0] - 1) // block_dim[0])

    matScale[grid_dim, block_dim](d_A, d_B, scale)

    # Copy result to host
    d_B.copy_to_host(h_B)

    # Display matrices and result
    print("Matrix A:")
    print(h_A)
    print("--------")
    print("Matrix B (Result of Scaling):")
    print(h_B)


Matrix A:
[[9. 5. 4. 5.]
 [8. 1. 3. 9.]
 [8. 5. 4. 9.]
 [1. 2. 7. 5.]]
--------
Matrix B (Result of Scaling):
[[3.6 2.  1.6 2. ]
 [3.2 0.4 1.2 3.6]
 [3.2 2.  1.6 3.6]
 [0.4 0.8 2.8 2. ]]


/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


In [53]:
import numpy as np
from numba import cuda, float32

@cuda.jit
def relu(d_a, d_b, alpha):
    col = cuda.grid(1)

    if col < d_b.shape[0]:
        d_b[col] = max(alpha * d_a[col], d_a[col])


if __name__ == "__main__":
    # Vector size
    N = 8

    # Leaky ReLU parameter
    alpha = 0.01


    h_a = np.random.randint(-5, 8, size=N).astype(np.float32)
    h_b = np.empty_like(h_a)

    # Allocate device memory
    d_a = cuda.to_device(h_a)
    d_b = cuda.to_device(h_b)

    block_dim = 256
    grid_dim = (N + block_dim - 1) // block_dim

    relu[grid_dim, block_dim](d_a, d_b, alpha)

    # Copy result to host
    d_b.copy_to_host(h_b)

    print("Vector A:")
    print(h_a)
    print("\nLeaky ReLU:")
    print(h_b)


/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


Vector A:
[-2.  6.  6.  2.  3. -5.  5.  5.]

Leaky ReLU:
[-0.02  6.    6.    2.    3.   -0.05  5.    5.  ]


In [54]:
import numpy as np
from numba import cuda, float32

@cuda.jit
def softmax(d_in, d_out):
    col = cuda.grid(1)

    if col < d_out.shape[0]:
        N = d_out.shape[0]

        local_exp = math.exp(d_in[col])

        # Allocate shared memory for parallel reduction
        shared_exp = cuda.shared.array(256, dtype=float32)

        # Store exp(x) in shared memory
        shared_exp[cuda.threadIdx.x] = local_exp

        # Synchronize!
        cuda.syncthreads()

        # Parallel reduction to compute sum(exp(x))
        for stride in range(cuda.blockDim.x // 2, 0, -1):
            if cuda.threadIdx.x < stride:
                shared_exp[cuda.threadIdx.x] += shared_exp[cuda.threadIdx.x + stride]

            # Synchronize threads after each step of reduction
            cuda.syncthreads()

        # Calculate softmax for the current element
        d_out[col] = local_exp / shared_exp[0]

# Main function
if __name__ == "__main__":
    # Vector size
    N = 8

    # Generate random input vector
    h_in = np.random.randint(1, 8, size=N).astype(np.float32)
    h_out = np.empty_like(h_in)

    # Allocate device memory
    d_in = cuda.to_device(h_in)
    d_out = cuda.to_device(h_out)

    block_dim = 256
    grid_dim = (N + block_dim - 1) // block_dim

    softmax[grid_dim, block_dim](d_in, d_out)

    # Copy result to host
    d_out.copy_to_host(h_out)

    print("Softmax input:")
    print(h_in)
    print("Softmax output:")
    print(h_out)


/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


Softmax input:
[2. 1. 6. 5. 7. 5. 4. 6.]
Softmax output:
[0.00111113 0.00040876 0.06066555 0.02231761 0.16490605 0.02231761
 0.00821019 0.06066555]


In [55]:
import numpy as np
from numba import cuda

@cuda.jit
def transpose_kernel(d_A, d_T, M, N):
    row, col = cuda.grid(2)

    if row < M and col < N:
        d_T[col, row] = d_A[row, col]

if __name__=="__main__":

  M = 4
  N = 4

  # Initialize host data
  h_A = np.random.randint(1, 11, size=(M, N)).astype(np.float32)
  h_T = np.empty((N, M), dtype=np.float32)

  # Allocate device memory
  d_A = cuda.to_device(h_A)
  d_T = cuda.to_device(h_T)

  block_dim = (16, 16)
  grid_dim = ((N + block_dim[0] - 1) // block_dim[0], (M + block_dim[1] - 1) // block_dim[1])

  transpose_kernel[grid_dim, block_dim](d_A, d_T, M, N)

  # Copy result back to host
  d_T.copy_to_host(h_T)

  print("Matrix A:")
  print(h_A)
  print("Transpose:")
  print(h_T)


Matrix A:
[[ 1.  5.  2.  5.]
 [ 6.  8.  3.  6.]
 [10.  3.  2.  2.]
 [ 4.  4.  2.  5.]]
Transpose:
[[ 1.  6. 10.  4.]
 [ 5.  8.  3.  4.]
 [ 2.  3.  2.  2.]
 [ 5.  6.  2.  5.]]


/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


In [56]:
import numpy as np
from numba import cuda, float32

@cuda.jit
def addVectors(d_a, d_b, d_c):
    col = cuda.grid(1)

    if col < d_c.shape[0]:
        d_c[col] = d_a[col] + d_b[col]

if __name__ == "__main__":
    N = 10

    h_a = np.arange(N, dtype=np.float32) - 3
    h_b = np.arange(N, dtype=np.float32)
    h_c = np.empty(N, dtype=np.float32)

    # Allocate device memory
    d_a = cuda.to_device(h_a)
    d_b = cuda.to_device(h_b)
    d_c = cuda.device_array_like(h_c)

    block_dim = 256
    grid_dim = (N + block_dim - 1) // block_dim

    addVectors[grid_dim, block_dim](d_a, d_b, d_c)

    # Copy result to host
    d_c.copy_to_host(h_c)

    for i in range(10):
        print(f'a: {h_a[i]} b: {h_b[i]} c: {h_c[i]}')





/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


a: -3.0 b: 0.0 c: -3.0
a: -2.0 b: 1.0 c: -1.0
a: -1.0 b: 2.0 c: 1.0
a: 0.0 b: 3.0 c: 3.0
a: 1.0 b: 4.0 c: 5.0
a: 2.0 b: 5.0 c: 7.0
a: 3.0 b: 6.0 c: 9.0
a: 4.0 b: 7.0 c: 11.0
a: 5.0 b: 8.0 c: 13.0
a: 6.0 b: 9.0 c: 15.0


In [57]:
import numpy as np
from numba import cuda

@cuda.jit
def  batch_matrix_multiplication(d_A, d_B, d_C, batch_size, M, N, P):
    row, col, batch = cuda.grid(3)

    if batch < batch_size and row < M and col < P:
        sum = 0.0

        for i in range(N):
            sum += d_A[batch, row, i] * d_B[batch, i, col]

        d_C[batch, row, col] = sum

if __name__ == "__main__":
    M = 2
    N = 3
    P = 5
    batch_size = 4

    h_A = np.random.randint(1, 11, size=(batch_size, M, N)).astype(np.float32)
    h_B = np.random.randint(1, 11, size=(batch_size, N, P)).astype(np.float32)
    h_C = np.empty((batch_size, M, P), dtype=np.float32)

    # Allocate device memory
    d_A = cuda.to_device(h_A)
    d_B = cuda.to_device(h_B)
    d_C = cuda.device_array_like(h_C)

    # Define block and grid dimensions
    block_dim = (8, 8, batch_size)
    grid_dim = ((P + block_dim[0] - 1) // block_dim[0], (M + block_dim[1] - 1) // block_dim[1], batch_size)

    batch_matrix_multiplication[grid_dim, block_dim](d_A, d_B, d_C, batch_size, M, N, P)

    # Copy result to host
    d_C.copy_to_host(h_C)

    for batch in range(batch_size):
        print(f"Matrix A (Batch {batch + 1}):\n", h_A[batch])
        print(f"Matrix B (Batch {batch + 1}):\n", h_B[batch])
        print(f"Matrix C (Batch {batch + 1}):\n", h_C[batch])
        print("--------")



/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 4 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


Matrix A (Batch 1):
 [[ 4.  1.  1.]
 [ 4.  6. 10.]]
Matrix B (Batch 1):
 [[ 6. 10.  5.  4.  3.]
 [ 8.  2.  1.  2.  7.]
 [ 8.  9. 10. 10.  8.]]
Matrix C (Batch 1):
 [[ 40.  51.  31.  28.  27.]
 [152. 142. 126. 128. 134.]]
--------
Matrix A (Batch 2):
 [[7. 6. 1.]
 [9. 7. 8.]]
Matrix B (Batch 2):
 [[ 6.  3.  4. 10.  8.]
 [ 3.  7.  5. 10.  9.]
 [ 7.  5.  2.  3.  4.]]
Matrix C (Batch 2):
 [[ 67.  68.  60. 133. 114.]
 [131. 116.  87. 184. 167.]]
--------
Matrix A (Batch 3):
 [[4. 4. 8.]
 [7. 7. 2.]]
Matrix B (Batch 3):
 [[ 8.  5.  6.  8. 10.]
 [ 9.  1.  6.  4.  8.]
 [ 5.  1.  8.  3.  9.]]
Matrix C (Batch 3):
 [[108.  32. 112.  72. 144.]
 [129.  44. 100.  90. 144.]]
--------
Matrix A (Batch 4):
 [[2. 5. 4.]
 [1. 7. 6.]]
Matrix B (Batch 4):
 [[ 1.  8.  4.  2.  7.]
 [ 3.  5.  3.  5.  6.]
 [ 8.  8. 10.  2.  2.]]
Matrix C (Batch 4):
 [[49. 73. 63. 37. 52.]
 [70. 91. 85. 49. 61.]]
--------
